In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('../data/train.csv')

In [6]:
import sweetviz as sv

# Assuming that data is a pandas DataFrame
report = sv.analyze(df)
report.show_html('report.html')

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:01 -> (00:00 left)


Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [19]:
df.sort_values(by=['target'], inplace=True, ascending=False)
df.head(20)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
2053019,56,190,510,1.576883e+06,1,1.001100,2.232349e+07,1.006211,1.003844,1.000721,86381.41,1.001100,28346.36,1.001006,446.07043,10501,190_510_56
2053217,56,190,520,1.697434e+06,1,1.000058,2.234029e+07,1.005643,1.003371,0.999585,21122.00,1.000437,3171.00,1.000326,392.88043,10502,190_520_56
5117011,31,470,0,5.874340e+06,1,0.997387,8.867263e+06,NaN,NaN,0.997387,10264.56,1.006136,24309.85,1.000000,387.77948,25850,470_0_31
1283586,5,119,530,2.913287e+06,1,0.951448,4.293521e+07,0.958473,0.956466,0.952874,18040.00,0.953138,440839.35,0.952884,357.43950,6598,119_530_5
2052821,56,190,500,1.538108e+06,1,1.001289,2.236227e+07,1.006211,1.004128,1.001005,52880.00,1.001573,4338.62,1.001530,352.00000,10500,190_500_56
2053415,56,190,530,0.000000e+00,0,1.007441,2.326307e+07,1.007441,1.007441,1.004034,21216.00,1.007252,372470.00,1.004208,325.62018,10503,190_530_56
1283389,5,119,520,3.064366e+06,1,0.951976,4.278413e+07,0.959318,0.955885,0.950867,108012.00,0.952663,9198.36,0.952522,298.29025,6597,119_520_5
5161011,31,474,0,1.201401e+07,1,0.989727,2.601458e+07,NaN,NaN,0.993639,28168.60,1.001463,6528.00,1.000000,268.87060,26070,474_0_31
2052623,56,190,490,1.764102e+06,1,1.000626,2.227362e+07,1.006211,1.003371,1.000437,12155.50,1.000626,25372.80,1.000498,268.80978,10499,190_490_56
1283783,5,119,540,0.000000e+00,0,0.971731,3.810749e+07,0.971731,0.971731,0.971361,31263.00,0.972629,103670.82,0.971655,261.31990,6599,119_540_5


In [ ]:
# see unique stock_id in df
df['stock_id'].unique()

# LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Assuming df is your DataFrame and you want to predict the 'target' column
target_column = 'target' # column you want to predict
n_steps = 3  # number of time steps you want to consider in your sequence

# Prepare your data
X, y = [], []
for i in range(n_steps, len(df)):
    X.append(df[i-n_steps:i].values)
    y.append(df.loc[i, target_column])
X, y = np.array(X), np.array(y)

# Reshape X to fit LSTM layer input shape requirements
X = X.reshape((X.shape[0], X.shape[1], df.shape[1]))

# Define LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, df.shape[1])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(X, y, epochs=200, verbose=0)

# Predict
X_test = np.array(df[-n_steps:].values).reshape((1, n_steps, df.shape[1]))